In [1]:
import tensorflow as tf
import os
import pickle 
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
         with open(filename,'rb') as f:
            data = pickle.load(f,encoding="bytes")
            return data[b'data'],data[b'labels']
        
class CifarData:
     def __init__(self,filenames,need_shuffle):
            all_data = []
            all_labels = []
            for filename in filenames:
                data,labels = load_data(filename)
                all_data.append(data)
                all_labels.append(labels)
            self._data = np.vstack(all_data)
            self._data = self._data/127.5-1
            self._labels = np.hstack(all_labels)
            
            self._num_examples = self._data.shape[0]
            self._need_shuffle = need_shuffle
            self._indicator = 0
            if self._need_shuffle:
                 self._shuffle_data()
                    
     def _shuffle_data(self):
             p = np.random.permutation(self._num_examples)
             self._data = self._data[p]
             self._labels = self._labels[p]
            
     def next_batch(self,batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels

train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d'% i) for i in range(1,6)]
test_data_filenames = [os.path.join(CIFAR_DIR,'test_batch')]
train_data = CifarData(train_filenames,True)
test_data = CifarData(test_data_filenames,False)

In [3]:
#计算图
x = tf.placeholder(tf.float32,[None,3072])
# [None], eg:[0,5,6,3]
y = tf.placeholder(tf.int64,[None])
# (3072,10)
w = tf.get_variable('w',[x.get_shape()[-1],10],
initializer = tf.random_normal_initializer(0,1))

#(10,)
b = tf.get_variable('b',[10],
initializer = tf.constant_initializer(0.0))
# [None,3072]*[3072,10] =[None,10]
y_=tf.matmul(x,w)+b
# course:1 + e^x
# api:e^x/sum(e^x)
# [0.01,0.9,....,0.03],等概率分布
p_y = tf.nn.softmax(y_)

# one-hot 编码将p_y 变成一个分布 eg: 5 ->[0,0,0,0,1,0,0,0,0,0]
y_one_hot = tf.one_hot(y,10,dtype=tf.float32)
# 平方差损失函数
# loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
# 交叉熵
loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
# y_-> softmax
# y -> one_hot
# ylogy

# index
predict = tf.argmax(y_,1)

correct_prediction = tf.equal(predict,y)

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

with tf.name_scope('train'):
      train_op = tf.train.AdadeltaOptimizer(1e-3).minimize(loss)
        
train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d'% i) for i in range(1,6)]
test_data_filenames = [os.path.join(CIFAR_DIR,'test_batch')]
train_data = CifarData(train_filenames,True)
test_data = CifarData(test_data_filenames,False)

In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000

test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        loss_val,acc_val,_=sess.run(
            [loss,accuracy,train_op],
            feed_dict={
                x:batch_data,
                y:batch_labels
            }
        )
        if (i+1)%500==0:
            print('[Train] Step:%d,loss:%4.5f,acc:%4.5f'\
                 %(i+1,loss_val,acc_val))
        if(i+1)%5000 == 0:
            test_data = CifarData(test_data_filenames,False)
            all_test_acc_val =[]
            for j in range(test_steps):
                test_batch_data,test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict={
                        x:test_batch_data,
                        y:test_batch_labels
                    }
                )
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('Test Step:%d,acc:%4.5f'%(i+1,test_acc))

[Train] Step:500,loss:44.00255,acc:0.15000
[Train] Step:1000,loss:40.63790,acc:0.05000
[Train] Step:1500,loss:39.39275,acc:0.15000
[Train] Step:2000,loss:43.16206,acc:0.05000
[Train] Step:2500,loss:32.22092,acc:0.10000
[Train] Step:3000,loss:26.33700,acc:0.20000
[Train] Step:3500,loss:31.68901,acc:0.10000
[Train] Step:4000,loss:30.13493,acc:0.20000
[Train] Step:4500,loss:38.48960,acc:0.10000
[Train] Step:5000,loss:43.53895,acc:0.25000
Test Step:5000,acc:0.11300
[Train] Step:5500,loss:37.58607,acc:0.15000
[Train] Step:6000,loss:38.69221,acc:0.10000
[Train] Step:6500,loss:33.53119,acc:0.15000
[Train] Step:7000,loss:40.30175,acc:0.05000
[Train] Step:7500,loss:40.20905,acc:0.05000
[Train] Step:8000,loss:34.64463,acc:0.05000
[Train] Step:8500,loss:40.60851,acc:0.00000
[Train] Step:9000,loss:28.84294,acc:0.05000
[Train] Step:9500,loss:32.83395,acc:0.10000
[Train] Step:10000,loss:43.34269,acc:0.05000
Test Step:10000,acc:0.11350
[Train] Step:10500,loss:40.96518,acc:0.20000
[Train] Step:11000,l

[Train] Step:86500,loss:35.59949,acc:0.05000
[Train] Step:87000,loss:38.50426,acc:0.10000
[Train] Step:87500,loss:31.28964,acc:0.25000
[Train] Step:88000,loss:38.87843,acc:0.10000
[Train] Step:88500,loss:37.56368,acc:0.10000
[Train] Step:89000,loss:34.06548,acc:0.05000
[Train] Step:89500,loss:32.88012,acc:0.15000
[Train] Step:90000,loss:27.53537,acc:0.20000
Test Step:90000,acc:0.13400
[Train] Step:90500,loss:36.90643,acc:0.10000
[Train] Step:91000,loss:40.19246,acc:0.05000
[Train] Step:91500,loss:29.36103,acc:0.20000
[Train] Step:92000,loss:26.11014,acc:0.20000
[Train] Step:92500,loss:36.14431,acc:0.00000
[Train] Step:93000,loss:22.47518,acc:0.20000
[Train] Step:93500,loss:25.98501,acc:0.20000
[Train] Step:94000,loss:50.78711,acc:0.05000
[Train] Step:94500,loss:38.19529,acc:0.05000
[Train] Step:95000,loss:25.24088,acc:0.15000
Test Step:95000,acc:0.13350
[Train] Step:95500,loss:24.70989,acc:0.20000
[Train] Step:96000,loss:35.87392,acc:0.00000
[Train] Step:96500,loss:31.90532,acc:0.15000